In [36]:
import gensim.downloader as api
wv = api.load("word2vec-google-news-300")

In [37]:
import pandas as pd
df=pd.read_csv("data.csv")
df

,Unnamed: 0,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Fake
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",True
2,2,Why the Truth Might Get You Fired,Fake
3,3,15 Civilians Killed In Single US Airstrike Hav...,Fake
4,4,Iranian woman jailed for fictional unpublished...,Fake
...,...,...,...
34319,34319,trump vs clinton risk vs disaster,Real
34320,34320,gingrich slutshames megyn kelly,Real
34321,34321,youtube bans clintons black son,Real
34322,34322,wikileaks bombshells hillary need know,Real


In [38]:
df = df.drop(["Unnamed: 0"],axis = 1)

In [39]:
df

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Fake
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",True
2,Why the Truth Might Get You Fired,Fake
3,15 Civilians Killed In Single US Airstrike Hav...,Fake
4,Iranian woman jailed for fictional unpublished...,Fake
...,...,...
34319,trump vs clinton risk vs disaster,Real
34320,gingrich slutshames megyn kelly,Real
34321,youtube bans clintons black son,Real
34322,wikileaks bombshells hillary need know,Real


In [40]:
wv.similarity(w1 = "great" ,w2 = "good")

0.72915095

In [41]:
wv_great = wv["great"]
wv_good = wv["good"]

In [42]:
wv_great.shape

(300,)

In [43]:
df["label"] = df["label"].replace("Real", "True")

In [44]:
df["label_number"] = df["label"].map({"Fake" : 0 , "True" : 1})

In [45]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [50]:
def preprocess_and_vectorize(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    
    if not filtered_tokens:
        return None
    
    return wv.get_mean_vector(filtered_tokens)

In [51]:
preeprocee_and_vectorize("Don't worry if you dont understand").shape

(300,)

In [52]:
wv.get_mean_vector("worry', 'not', 'understand").shape # it take out the mean od 2 vectors and go for next

(300,)

In [53]:
df['vector'] = df['text'].apply(lambda text: preprocess_and_vectorize(text))

In [54]:
df

,text,label,label_number,vector
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Fake,0,"[0.0014937663, 0.020068808, -0.04331702, 0.012..."
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",True,1,"[0.016189992, 0.020764446, -0.004613613, 0.023..."
2,Why the Truth Might Get You Fired,Fake,0,"[0.11566126, 0.005249493, 0.061008282, -0.0377..."
3,15 Civilians Killed In Single US Airstrike Hav...,Fake,0,"[0.013792393, -0.014312416, 0.008747155, 0.030..."
4,Iranian woman jailed for fictional unpublished...,Fake,0,"[0.016219957, -0.0026730283, 0.014577476, 0.01..."
...,...,...,...,...
34319,trump vs clinton risk vs disaster,True,1,"[0.026183164, 0.049257915, 0.013204667, 0.0570..."
34320,gingrich slutshames megyn kelly,True,1,"[-0.06286589, 0.021545585, -0.03895914, 0.0808..."
34321,youtube bans clintons black son,True,1,"[0.01061044, 0.0010857662, 0.034597438, -0.004..."
34322,wikileaks bombshells hillary need know,True,1,"[0.025162108, -0.011468755, 0.0071123666, 0.00..."


In [55]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test  = train_test_split(df["vector"] , df["label_number"] , test_size=0.2, random_state=2023,stratify=df.label_number)

In [57]:
X_train.shape,X_test.shape

((27459,), (6865,))

In [68]:
X_train[1]

array([ 0.01618999,  0.02076445, -0.00461361,  0.02306211,  0.00531394,
       -0.04865761, -0.02393252,  0.0408179 ,  0.02122347, -0.01092498,
       -0.04892654,  0.02433237, -0.00836643,  0.01835289, -0.02384271,
       -0.01158173, -0.0225379 , -0.00900037,  0.0312991 , -0.00511046,
        0.0062368 ,  0.0246229 ,  0.03283746, -0.00136622, -0.00343243,
       -0.01471622, -0.0799661 , -0.02020641,  0.08880306, -0.02582435,
       -0.01704667, -0.01641476, -0.08260082, -0.07276283, -0.00049194,
       -0.01785448,  0.02727868, -0.0083852 , -0.00097104,  0.03056455,
        0.00068175, -0.01230871, -0.00127236, -0.01005715, -0.04280764,
       -0.0047126 , -0.00346063,  0.00075429,  0.02585504,  0.05003463,
       -0.04204959, -0.00415322, -0.01319373, -0.02124206,  0.01667621,
        0.00298622,  0.0066951 , -0.05974533,  0.00899359, -0.09018335,
        0.05070996,  0.0299376 , -0.03265302, -0.01618041,  0.0403474 ,
        0.00646648, -0.03551447,  0.021754  , -0.02570143, -0.00

In [60]:
import numpy as np


In [71]:


X_train_2d = np.stack(X_train)
X_test_2d =  np.stack(X_test)



ValueError: all input arrays must have the same shape

In [65]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = max([len(x) for x in X_train])
X_train = pad_sequences(X_train, maxlen=max_len, padding='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post')

TypeError: object of type 'NoneType' has no len()

In [70]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

#1. creating a GradientBoosting model object
clf = GradientBoostingClassifier()

#2. fit with all_train_embeddings and y_train
clf.fit(X_train, y_train)


#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(X_test_2d)


#4. print the classfication report
print(classification_report(y_test, y_pred))

ValueError: setting an array element with a sequence.